# Data Augmentation

In [198]:
import albumentations as A
import cv2

baby_monitor_type_filter = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(-0.4, -0.2), contrast_limit=(-0.3, 0), p=1.0),
    A.MotionBlur(blur_limit=(5, 15), p=0.6),
    A.GaussianBlur(blur_limit=(5, 11), p=0.4),
    A.MedianBlur(blur_limit=7, p=0.3),
    A.GaussNoise(p=0.6),
    A.ISONoise(color_shift=(0.01, 0.03), intensity=(0.1, 0.4), p=0.3),
    A.HueSaturationValue(hue_shift_limit=(-2, 2), sat_shift_limit=(-25, -10), val_shift_limit=(-10, 0), p=0.5),
    A.Affine(rotate=(-15, 15), p=0.5)
])

gray_filter = A.Compose([
    A.ToGray(p=1.0),
    A.Affine(rotate=(-90, 90), p=0.5)
])

filter_3 = A.Compose([
    A.ToGray(p=1.0),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5),
    A.ImageCompression(p=0.4),
    A.MotionBlur(blur_limit=(3, 7), p=0.4),
    A.GaussNoise(p=0.3),
    A.Affine(rotate=(-90, 90), p=0.5)
])

filter_4 = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.05), contrast_limit=(-0.2, 0.05), p=1.0),
    A.CLAHE(clip_limit=(1, 3), tile_grid_size=(8, 8), p=0.4),
    A.Equalize(mode='cv', p=0.3),
    A.GaussianBlur(blur_limit=(3, 7), p=0.4),
    A.ImageCompression(p=0.3),
    A.Affine(translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)}, rotate=(-3, 3),p=0.3),
    A.Affine(rotate=(-90, 90), p=0.5)
])

night_aug_list = [baby_monitor_type_filter, gray_filter,filter_3,filter_4]

In [199]:
def apply_transformation(image_input, image_output,mode):
    image = cv2.imread(image_input)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    augmented = night_aug_list[mode](image=image)["image"]
    augmented_bgr = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_output, augmented_bgr)

In [200]:
from pathlib import Path
import re
import shutil

def clean_dataset(dir):

    dir_images = Path(dir+"/images")
    dir_labels = Path(dir+"/labels")

    images = sorted(f for f in dir_images.iterdir() if f.is_file())
    labels = sorted(f for f in dir_labels.iterdir() if f.is_file())

    for image in images:
        if re.search(r"_t\d+\.jpg$", image.name):
                image.unlink(missing_ok=True)

    for label in labels:
        if label:
            if re.search(r"_t\d+\.txt$", label.name):
                label.unlink(missing_ok=True)

def add_items_in_dataset(dir,mode):
    dir_images = Path(dir+"/images")
    dir_labels = Path(dir+"/labels")

    images = sorted(f for f in dir_images.iterdir() if f.is_file() and not re.search(r"_t\d+\.jpg$", f.name))
    labels = sorted(f for f in dir_labels.iterdir() if f.is_file() and not re.search(r"_t\d+\.txt$", f.name))

    for image in images:
         if image:
            image_name = str(image).split(".jpg")[0]
            apply_transformation(str(image),str(image_name)+"_t"+str(mode)+".jpg",mode)
    for label in labels:
        if label:
            label_name = str(label).split(".txt")[0]
            shutil.copy2(label, Path(label_name+"_t"+str(mode)+".txt"))

In [201]:
#clean_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/test")

In [203]:
#Data augmentation Test
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/test",0)
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/test",1)
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/test",2)
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/test",3)

#Data augmentation Train
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/train",0)
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/train",1)
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/train",2)
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/train",3)

#Data augmentation Test
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/valid",0)
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/valid",1)
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/valid",2)
add_items_in_dataset("/Users/lorenzodimaio/PyCharmMiscProject/second_try_face_detection.v1i.yolov8/valid",3)

KeyboardInterrupt: 